In [6]:
from langchain_groq import ChatGroq
import json
import os
from itertools import chain

In [7]:
from dotenv import load_dotenv

load_dotenv()

True

In [64]:
llm = ChatGroq(temperature=0.2, model_name="llama3-70b-8192")

In [113]:
template = """
I am building a dataset for Named Entity Recognition (NER) for a job recommender system with 4 entities: Job Role, Domain, Experience Level, and Skill. Here is an example:

{
    "tokens": [
      "DevOps",
      "Engineer",
      "needed",
      "for",
      "gaming",
      "company",
      "with",
      "4+",
      "years",
      "of",
      "experience",
      "in",
      "Kubernetes",
      "and",
      "Docker"
    ],
    "labels": [
      "B-Job_Role",
      "B-Job_Role",
      "O",
      "O",
      "B-Domain",
      "B-Domain",
      "O",
      "B-Experience_Level",
      "B-Experience_Level",
      "O",
      "O",
      "O",
      "B-Skill",
      "O",
      "B-Skill"
    ]
  }

Please provide me with 20 more realistic and diverse instances, including various skills (both popular and less popular programming languages  frameworks(react flutter spring react native vue angular .NET etc), and technologies), job roles, and domains and experience levels that I can use to train an NER model (make them look as real as possible).

Return only the JSON format.
"""

In [114]:
def extract_json_data(items):
    json_data = []
    for item in items:
        document_content = item
        if isinstance(document_content, str):
            start_index = document_content.find("[")
            end_index = document_content.rfind("]") + 1
            if start_index != -1 and end_index != -1:
                json_string = document_content[start_index:end_index]
                try:
                    data = json.loads(json_string)
                    json_data.append(data)
                except json.JSONDecodeError:
                    print(
                        f"Error: Invalid JSON in document content: {document_content}"
                    )
    return json_data

In [115]:
def generate_new_data(llm, template):
    output = llm.invoke(template)
    dataset = [output.content]
    datasets = extract_json_data(dataset)
    datasets = list(chain(*datasets))
    return datasets

In [116]:
def append_to_json_file(json_array_data, file_path):
    with open(file_path, "r") as file:
        data = json.load(file)
    if isinstance(data, list):
        for element in json_array_data:
            data.append(element)
    else:
        print("Invalid JSON data format")
    with open(file_path, "w") as file:
        json.dump(data, file, indent=4)

In [117]:
datasets = generate_new_data(llm, template)

In [118]:
datasets

[{'tokens': ['Senior',
   'Software',
   'Engineer',
   'required',
   'for',
   'fintech',
   'company',
   'with',
   '7-10',
   'years',
   'of',
   'experience',
   'in',
   'Java',
   'and',
   'Spring'],
  'labels': ['B-Job_Role',
   'B-Job_Role',
   'B-Job_Role',
   'O',
   'O',
   'B-Domain',
   'B-Domain',
   'O',
   'B-Experience_Level',
   'B-Experience_Level',
   'O',
   'O',
   'B-Skill',
   'O',
   'B-Skill']},
 {'tokens': ['Full',
   'Stack',
   'Developer',
   'needed',
   'for',
   'healthcare',
   'startup',
   'with',
   '3-5',
   'years',
   'of',
   'experience',
   'in',
   'React',
   'and',
   'Redux'],
  'labels': ['B-Job_Role',
   'B-Job_Role',
   'B-Job_Role',
   'O',
   'O',
   'B-Domain',
   'B-Domain',
   'O',
   'B-Experience_Level',
   'B-Experience_Level',
   'O',
   'O',
   'B-Skill',
   'O',
   'B-Skill']},
 {'tokens': ['Data',
   'Scientist',
   'position',
   'available',
   'at',
   'tech',
   'giant',
   'with',
   '5+',
   'years',
   'of',
   'e

In [105]:
append_to_json_file(datasets, "datasets/ner-large-dataset-train.json")